In [1]:
import numpy as np
import pandas as pd

from datetime import datetime

In [2]:
def read_df(fname):
    df = pd.read_csv('companies.csv', parse_dates=['branch_issue_deadline', 'exam_date', 'interview_date', 'ppt_date', 'updated_at', 'willingness_deadline'])
    return df

In [3]:
def query_dept_dates(df, dept, date_col, min_date=None):
    if min_date is None:
        min_date = datetime.now()
    return df[(df[dept] == 1) & (df[date_col] > min_date)].copy()

In [4]:
def create_notification_df(df, dept, date_col, min_date=None, suffix=None):
    notification_df = query_dept_dates(df, dept, date_col, min_date)
    
    if suffix is None:
        suffix = date_col
    notification_df.loc[:, 'email_msg_{}'.format(suffix)] = create_emails(notification_df)
    notification_df.loc[:, 'email_count_{}'.format(suffix)] = 0
    return notification_df.reset_index(drop=True)

In [5]:
def email(x):
    company = x['company_name']
    package_ctc = x['idd_imd_ctc'] / 10**5
    package_home = x['idd_imd_home'] / 10**5
    willingness = x['willingness_deadline']
    msg = ('Willingness for {} is open till {}. '
           'The Company is offering a CTC of {:.2f} Lakhs and Take Home of {:.2f} Lakhs'
           .format(company, willingness, package_ctc, package_home))
    return msg

In [6]:
def create_emails(df):
    return df.apply(email, axis=1)

In [7]:
df = read_df('companies.csv')
willingness_df = create_notification_df(df, 'mat', 'willingness_deadline', suffix='willingness')
willingness_df.loc[:, ['company_name', 'email_msg_willingness']]

,company_name,email_msg_willingness
0,VMWare,Willingness for VMWare is open till 2017-09-22...
1,INTEL,Willingness for INTEL is open till 2017-09-22 ...
2,GGK Tech,Willingness for GGK Tech is open till 2017-09-...
3,AXTRIA,Willingness for AXTRIA is open till 2017-09-22...


In [8]:
willingness_df.to_csv('willingness.csv', index=False)
pd.read_csv('willingness.csv')

,active_backlog,branch_issue_deadline,btech_ctc,btech_home,cgpa,company_name,exam_date,idd_imd_ctc,idd_imd_home,interview_date,...,cse,ece,mst,phe,btech,idd,mtech,phd,email_msg_willingness,email_count_willingness
0,0.0,2017-09-06 12:00:00,2202108.0,NaN,7.0,VMWare,2017-10-13 00:00:00,2202108.0,NaN,NaN,...,1,1,0,0,1,1,0,0,Willingness for VMWare is open till 2017-09-22...,0
1,0.0,2017-09-06 12:00:00,NaN,NaN,7.0,INTEL,NaN,1709140.0,1086000.0,NaN,...,1,1,0,0,0,1,1,0,Willingness for INTEL is open till 2017-09-22 ...,0
2,0.0,2017-09-06 12:00:00,800000.0,NaN,5.0,GGK Tech,2017-09-25 00:00:00,800000.0,NaN,NaN,...,1,1,1,1,1,1,0,0,Willingness for GGK Tech is open till 2017-09-...,0
3,0.0,2017-09-01 11:00:00,NaN,NaN,6.0,AXTRIA,NaN,780000.0,NaN,NaN,...,1,0,1,1,0,1,0,0,Willingness for AXTRIA is open till 2017-09-22...,0
